## Lecture 15 EEP118

In [ ]:
#install packaged and load the data
#Lecture15.R
#LECTURE 15

# Load the 'pacman' package
library(pacman) 
#packages to use load them now using the pacman "manager"
p_load(dplyr, haven, readr) 
#Another great feature of p_load(): if you try to load a package that is not installed on your machine, p_load() install the package for you, rather than throwing an error. For instance, let’s install and load one final package named ggplot2. 
p_load(ggplot2)

pacman::p_load(lfe, lmtest, haven, sandwich, tidyverse)
# lfe for running fixed effects regression
# lmtest for displaying robust SE in output table
# haven for loading in dta files
# sandwich for producing robust Var-Cov matrix
# tidyverse for manipulating data and producing plots


#if in R studio comment the following line to change into Lecture 15 directory
#setwd("/Users/sofiavillas-boas/Dropbox/EEP118_Spring2023/Lectures/Lecture15")

#set scientific display off, thank you Roy
options(scipen=999)

#read in a Stata dataset
my_data <- read_dta("Lecture14hprice1.dta")
head(my_data)

#    Variable |       Obs        Mean    Std. Dev.       Min        Max
#-------------+--------------------------------------------------------
#  price |      88     293.546    102.7134        111        725 	 price       house price, $1000s
#bdrms |        88    3.568182    .8413926          2          7	 bdrms        number of bedrooms
#lotsize |      88    9019.864    10174.15       1000      92681	 lotsize       size of lot in square feet 
#sqrft |        88    2013.693    577.1916       1171       3880	 sqrft       size of house in square feet
#colonial |     88    .6931818    .4638161          0          1		colonial     =1 if home is colonial style




### How to obtain an estimate of the Prediction for the average value of all houses with bdrms=3, sqrft=2000, and lotsize=9000, that is get a prediction for the value of a house on average with certain characteristics?



The trick is to transform the data and run a regression with the transformed data so that then the estimate of the constant is the average prediction you want and you also gets its standard error

In [ ]:

#trick
#Prediction for the average value of all houses with bdrms=3, sqrft=2000, and lotsize=9000:
#generate transfored variables such that then the estimated constant gives us what we need
#gen bdrms0=bdrms-3
#gen sqrft0=sqrft-2000
#gen lotsize0=lotsize-9000
#reg price bdrms0 sqrft0 lotsize0

my_data$bdrms0<-my_data$bdrms-3
my_data$sqrft0<-my_data$sqrft-2000
my_data$lotsize0<-my_data$lotsize-9000
reg14rev <- lm(price~bdrms0+lotsize0+sqrft0, my_data)
summary(reg14rev)


From the output we can get a confidence interval

t table got 5% critical from the t 88-4=84 degrees of freedom,, tc14


95% conf interval for predicted average prices of a 3 bd rooom house 2000 sqrt, 9000lot

2.840e+02- tc14 * 8.121e+00 ;   2.840e+02+ tc14 * 8.121e+00

The critical value for the 95% confidence interval is 2 (between 1.987 and 2.000 to be exact).
The CI for average price E is thus

 283.95 ±2 (8.12)  ≈   267.7,   300.2  .  
 
We predict the mean price to be between $267,700 and $300,200. This was the CI for the average house E (of the above characteristics).
The CI for the average house E is not the same as the CI for p, the price of a particular house of the above characteristics!!!


In [ ]:
#regression  in logs and in levels
# and how to choose:

#in logs
reg15log <- lm(lprice~bdrms+lotsize+sqrft, my_data)
summary(reg15log)
lprice_hat<-reg15log$fitted.values
price_hat<-exp(lprice_hat)
aa<-exp(0.1899*0.1899*0.5)
my_data$price_hat<-price_hat*aa
#correlate then
cor(my_data$price,my_data$price_hat)

In [ ]:

#in levels now:
reg15lev <- lm(price~bdrms+lotsize+sqrft, my_data)
summary(reg15lev)
my_data$price_hat2<-reg15lev$fitted.values
#correlate them
cor(my_data$price,my_data$price_hat2)

 We pick the one with the biggest correlation between price and predicted price.
 
 In this case:
 
 #correlate them
 cor(my_data$price,my_data$price_hat2)
[1] 0.819977   with levels


cor(my_data$price,my_data$price_hat)
[1] 0.8372371   with logs


If reg in levels, the correlation between price and predicted price is 0.8199

 If in logs , correlation of price and resulting predicted price is 0.8372, so…
 
 
 in logs is chosen